In [ ]:
import mysql.connector

db = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="admin",
    database="granddebattest"
)

cursor = db.cursor(dictionary=True)

In [ ]:
cursor.execute("""
SELECT th.nom_Theme, arg.texte, arg.type 
FROM `contribution` AS contr JOIN `arguments` AS arg USING(id_contrib) JOIN `theme` AS th ON contr.FK_theme_id = th.id_theme
""")

In [ ]:
arguments = cursor.fetchall()

In [ ]:
arguments = pd.DataFrame(arguments, columns=arguments[0].keys())

In [ ]:
arguments.to_csv("data/argumentsVD.csv", index=None, header=True)

In [ ]:
import pandas as pd
import os
import string
from collections import Counter
from time import time
import numpy as np
import itertools
from pandarallel import pandarallel
import matplotlib.pyplot as plt

In [ ]:
from textblob import TextBlob
import nltk
from nltk import word_tokenize
from nltk.util import ngrams

In [ ]:
import gensim
import gensim.corpora as corpora
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

In [ ]:
import spacy
nlp = spacy.load("fr_core_news_sm")

In [ ]:
seed = 1

In [ ]:
dates_GDN = ["31-01-19","06-02-19","17-02-19","02-03-19","08-03-19","21-03-19"]
sujets_GDN = {"ecologie": "LA_TRANSITION_ECOLOGIQUE",
             "democratie": "DEMOCRATIE_ET_CITOYENNETE",
             "fisc": "LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES",
             "etat": "ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS"}
sujets_VD = os.listdir("data/VD")

In [ ]:
#flatten = lambda l: [item for sublist in l for item in sublist]

In [ ]:
#-> Plus efficace
flatten = lambda l: list(itertools.chain(*l))

In [ ]:
filter_flatten_tags = lambda docs: [t for tokens in docs for t in tokens if not (t.is_punct or t.is_space or t.is_stop)]

In [ ]:
pd.set_option("display.max_columns",70)

______________________________

In [ ]:
df = pd.read_csv(f"data/GDN/{sujets_GDN['etat']}.csv")

In [ ]:
t = nlp(df[df.columns[-1]].dropna().values[500])

In [ ]:
resume = pd.read_csv("data/GDN/CR_RIL.csv")

In [ ]:
id_colonnes_utiles = [3,4,5,6,9,10,12,13,22,23,24]

In [ ]:
set(resume[resume.columns[-1]].dropna())

In [ ]:
resume[resume[resume.columns[-1]] != "Le document envoyé n'est pas un compte rendu "]

In [ ]:
resume[resume.Ville == "ARGENTEUIL"]

In [ ]:
resume.head()

In [ ]:
core_resume = resume[resume.columns[id_colonnes_utiles]]

In [ ]:
core_resume.columns

In [ ]:
pandarallel.initialize()

In [ ]:
test = core_resume[core_resume.columns[6]].dropna().str.lower().str.split(" ", n=1).str[0]

In [ ]:
test = core_resume[((core_resume[core_resume.columns[6]].str.lower().str.split(" ", n=1).str[0] != "cf") | (core_resume[core_resume.columns[6]].str.lower().str.split(" ", n=1).str[0] != "voir")) & (core_resume[core_resume.columns[6]].str.len() < 10)]#.str.lower().str.split(" ", n=1).str[0]

In [ ]:
df

In [ ]:
test

In [ ]:
set(test[test.columns[6]])

In [ ]:
test = [mot for mot in test if not (mot.startswith("cf") or mot.startswith("voir"))]

In [ ]:
all_constats = list(nlp.pipe(core_resume[core_resume.columns[6]].dropna()))

In [ ]:
all_props = list(nlp.pipe(core_resume[core_resume.columns[7]].dropna()))

In [ ]:
print(all_constats[5])

In [ ]:
filtered_constats = [(token.lemma_, token.pos_) for doc in all_constats for token in doc if not (token.is_punct or token.is_space or token.is_stop)]

In [ ]:
filtered_props = [(token.lemma_, token.pos_) for doc in all_props for token in doc if not (token.is_punct or token.is_space or token.is_stop)]

In [ ]:
constats_lemmas = [lemma for lemma, _ in filtered_constats]

In [ ]:
props_lemmas = [lemma for lemma, _ in filtered_props]

In [ ]:
def show_first_ngrams(data, n, nb):
    n_grams = ngrams(data, n)
    print(Counter(n_grams).most_common(nb))

In [ ]:
show_first_ngrams(constats_lemmas, 2, 10)

In [ ]:
Counter(filtered_props).most_common(10)

In [ ]:
resume.shape

In [ ]:
list(sorted(resume[resume.columns[0]]))[-1]

In [ ]:
for token in t.doc:
    if not (token.is_punct or token.is_space or token.is_stop):
        print(token.pos_, token.lemma_)

In [ ]:
df

In [ ]:
df.drop(27086, inplace=True)

In [ ]:
reg = df.groupby("region")
author = df.groupby("authorType")

region_test = reg.get_group('Corse')

In [ ]:
for group in author.groups:
    print(group, author.get_group(group).shape[0])

In [ ]:
reg.get_group(list(reg.groups.keys())[1])

In [ ]:
df.drop(author.get_group(list(author.groups.keys())[1]).index, inplace=True)

In [ ]:
get_min_rows = lambda groups: min([groups.get_group(group).shape[0] for group in groups.groups])

In [ ]:
def get_sampled_df(groups):
    sampled_dfs = []

    min_group = get_min_rows(groups)


    for group in groups.groups:
        group_data = groups.get_group(group)
        sampled_dfs.append(group_data.loc[np.random.RandomState(seed=seed).permutation(group_data.index)[:min(group_data.shape[0], 1000)]])

    sampled_df = pd.concat(sampled_dfs)
    return sampled_df
    


In [ ]:
pd.concat([reg, author], axi)

In [ ]:
sampled_reg = get_sampled_df(reg)

In [ ]:
sample = sampled_reg[[sampled_reg.columns[-1]]].dropna().reset_index(drop=True)

In [ ]:
pandarallel.initialize()

In [ ]:
sample['tokens'] = sample[sample.columns[0]].apply(nlp)

In [ ]:
sample

In [ ]:
get_sampled_df(reg).to_csv("data/sampled_regions.csv", index=None, header=True)

In [ ]:
get_sampled_df(author).to_csv("data/sampled_authors.csv", index=None, header=True)

In [ ]:
regions = pd.read_csv("data/sampled_regions.csv")

In [ ]:
regions.shape

In [ ]:
authors = pd.read_csv("data/sampled_authors.csv")

In [ ]:
authors.shape

In [ ]:
corse = reg.get_group('Corse')

In [ ]:
reg_print = df[["region"]].groupby("region")

for name, group in reg_print:
    print(name)
    print(group.shape)
    print("="*50)

In [ ]:
reg.describe()

________________

In [ ]:
def is_question_fermee(df,column, debug=False):
    nb_answers = len(df[column].unique())
    if debug:
        print(column)
        print("-"*20)
        print(f"Taille: {nb_answers} ==> {'FERMEE' if nb_answers < 20 else 'OUVERTE'}")
        print("="*50)
    return nb_answers < 20

In [ ]:
def remove_first_indices(df):
    return df.drop(df.columns[:6], axis=1)

In [ ]:
def remove_questions_fermees(df):
    return df.drop(df.columns[[i for i,column in enumerate(df.columns) if is_question_fermee(df,column) or i<6]], axis=1)

----------------------------

In [ ]:
def init_df(df):
    #indexes = ['docs', 'nb_prop', 'question_freq', 'tokens', 'pos', 'propLen_moy', 'nb_phrases_moy', 'unigram', 'bigram']
    indexes = ['nb_prop', 'question_freq', 'propLen_moy', 'nb_phrases_moy', 'unigram', 'bigram']
    
    questions_only = remove_first_indices(df)
    #ouvertes_only = remove_questions_fermees(df)

    new_df = pd.DataFrame(index=questions_only.columns, columns=indexes)
    
    return new_df, questions_only

In [ ]:
set().union(["abc", "def"])

In [ ]:
from nltk.tag.stanford import StanfordPOSTagger

In [ ]:
pos_tagger = StanfordPOSTagger("stanford/models/french.tagger", "stanford/stanford-postagger.jar", encoding='utf8') #instance de la classe StanfordPOSTagger en UTF-8

In [ ]:
from nltk.corpus import stopwords 

stops = set(stopwords.words('french')).union(string.punctuation+"’").union(["...", "--"])

def fill_df(df, data):
    #Listes pour pouvoir les insérer dans la dataframe
    for i,index in enumerate(df.index):
        df.loc[index]['nb_prop'] = data[index].dropna().shape[0]  
        df.loc[index]['question_freq'] = round(df.loc[index]['nb_prop']/data[index].shape[0], 2)
        #df.loc[index]['docs'] = list(nlp.pipe(data[index].dropna()))
        #df.loc[index]['tokens'] = filter_flatten_tags(df.loc[index]['docs'])
        #df.loc[index]['pos'] = [(token.lemma_, token.pos_) for token in df.loc[index]['tokens']]
        
        if not is_question_fermee(data, index):
            #df.loc[index]['propLen_moy'] = sum([len(doc) for doc in df.loc[index]['docs']])/df.loc[index]['nb_prop']
            df.loc[index]['propLen_moy'] = sum(data[data.columns[i]].dropna().apply(lambda prop: len(prop.split())))/df.loc[index]['nb_prop']
            #df.loc[index]['nb_phrases_moy'] = sum([len(TextBlob(doc.text).sentences) for doc in df.loc[index]['docs']])/df.loc[index]['nb_prop']
            df.loc[index]['nb_phrases_moy'] = sum([len(TextBlob(prop).sentences) for prop in data[data.columns[i]].dropna()])/df.loc[index]['nb_prop']

            lemmas = flatten([nltk.word_tokenize(prop.lower()) for prop in data[index].dropna().values])
            lemmas = [lemma for lemma in lemmas if lemma not in stops]
            #if(i==1):
                #print(sorted(lemmas))
            df.loc[index]['unigram'] = Counter(ngrams(lemmas, 1)).most_common(5)
            df.loc[index]['bigram'] = Counter(ngrams(lemmas, 2)).most_common(5)
        else:
            
            df.loc[index]['propLen_moy'] = "question fermée"
            df.loc[index]['nb_phrases_moy'] = "question fermée"
            
            df.loc[index]['unigram'] = "question fermée"
            df.loc[index]['bigram'] = "question fermée"
            
        #print(f"Question {i+1}/{len(df.index)} done!")


In [ ]:
"""
def fill_df(df, data):
    nb_prop = []
    docs = []
    tokens = []
    pos = []
    propLen_moy = []
    nb_phrases_moy = []
    unigram = []
    bigram = []

    #Listes pour pouvoir les insérer dans la dataframe
    for col in data.columns:
        nb_prop.append(data[col].shape[0])
        docs.append(list(nlp.pipe(data[col].dropna())))
        #docs -> tokens
        tokens.append(filter_flatten_tags(docs[-1]))
        #tokens -> pos
        pos.append([(token.lemma_, token.pos_) for token in tokens[-1]])
        #docs & nb_prop -> propLen_moy
        propLen_moy.append(sum([len(doc) for doc in docs[-1]])/nb_prop[-1])
        #docs & nb_prop -> nb_phrases_moy
        nb_phrases_moy.append(sum([len(TextBlob(doc.text).sentences) for doc in docs[-1]])/nb_prop[-1])
        #pos -> unigram
        unigram.append(list(ngrams([lemma for lemma, _ in pos[-1]], 1)))
        #pos -> bigram
        bigram.append(list(ngrams([lemma for lemma, _ in pos[-1]], 2)))
        
        print(f"Col {len(nb_prop)}/{len(data.columns)} done!")

    #Nombre de propositions
    df.loc['nb_prop'] = nb_prop

    #Chaque proposition sous forme de Doc spacy
    df.loc['docs'] = docs

    #Liste de tokens de toutes les propositions combinées
    df.loc['tokens'] = tokens

    #POS tag + lemme pour chaque token
    df.loc['POS'] = pos

    #Longueur moyenne d'une proposition
    df.loc['propLen_moy'] = propLen_moy

    #Nombre de phrases moyen pour une proposition
    df.loc['nb_phrases_moy'] = nb_phrases_moy

    #Unigram sur les tokens
    df.loc['unigram'] = unigram

    #Bigram sur les tokens
    df.loc['bigram'] =  bigram
"""
print()

In [ ]:
def get_df(df):
    new_df, data = init_df(df)
    fill_df(new_df, data)
    return new_df

In [ ]:
def get_regions_dfs(df):
    dfs = dict()
    
    reg = df.groupby("region")
    for i,region in enumerate(reg.groups):
        if region not in {"Normandie", "Corse"}:
            new_df = get_df(reg.get_group(region))
            new_df.to_csv(f"data/GDN_saves/{region}_INV.csv", header=True)
            
            print("="*50)
            print(f"({i+1}/{len(list(reg.groups))}) {region} TERMINEE", end=" ")
            print("="*50)
        
    

In [ ]:
_, corse = init_df(region_test)

In [ ]:
corse[corse.columns[0]].values

In [ ]:
import timeit

In [ ]:
from nltk import word_tokenize

In [ ]:
setup = """
import nltk
from __main__ import corse
"""

In [ ]:
corse

In [ ]:
save = calc_all_docs(df)

In [ ]:
pickle.dump(save, open("data/save.pkl", "wb"))

In [ ]:
test = pd.read_csv("data/GDN_saves/Corse_INV.csv")

In [ ]:
test['docs'].apply(lambda x: type(x[0]))

In [ ]:
new = pickle.load(open("data/save.pkl", "rb"))

In [ ]:
new['docs'].apply(lambda x: type(x[0]))

In [ ]:
save

In [ ]:
get_regions_dfs(df)

In [ ]:
def calc_docs(row):
    return nlp.pipe(data[row.name].dropna())

In [ ]:
from pandarallel import pandarallel

In [ ]:
from nltk.stem.snowball import FrenchStemmer
stemmer = FrenchStemmer()

In [ ]:
stemmer.stem("problematique")

In [ ]:
pandarallel.initialize()

In [ ]:
def fill_row(row):
    row.nb_prop = data[row.name].shape[0]  
    row.tokens = filter_flatten_tags(row.docs)
    row.pos = [(token.lemma_, token.pos_) for token in row.tokens]
    row.propLen_moy = sum([len(doc) for doc in row.docs])/row.nb_prop
    row.nb_phrases_moy = sum([len(TextBlob(doc.text).sentences) for doc in row.docs])/row.nb_prop
    row.unigram = list(ngrams([lemma for lemma, _ in row.pos], 1))
    row.bigram = list(ngrams([lemma for lemma, _ in row.pos], 2))
        
    return row
    #print(f"Question {i+1}/{len(df.index)} done!

In [ ]:
def fill_row(row):
    #Listes pour pouvoir les insérer dans la dataframe
    row.nb_prop = data[row.name].dropna().shape[0]  
    row.question_freq = round(row.nb_prop/data[row.name].shape[0], 2)
    #df.loc[index]['docs'] = list(nlp.pipe(data[index].dropna()))
    #df.loc[index]['tokens'] = filter_flatten_tags(df.loc[index]['docs'])
    #df.loc[index]['pos'] = [(token.lemma_, token.pos_) for token in df.loc[index]['tokens']]

    if not is_question_fermee(data, row.name):
        #df.loc[index]['propLen_moy'] = sum([len(doc) for doc in df.loc[index]['docs']])/df.loc[index]['nb_prop']
        row.propLen_moy = sum(data[row.name].dropna().apply(lambda prop: len(prop.split())))/row.nb_prop
        #df.loc[index]['nb_phrases_moy'] = sum([len(TextBlob(doc.text).sentences) for doc in df.loc[index]['docs']])/df.loc[index]['nb_prop']
        row.nb_phrases_moy = sum([len(TextBlob(prop).sentences) for prop in data[row.name].dropna()])/row.nb_prop
        
        #docs = filter_flatten_tags(nlp.pipe(data[row.name].dropna()))
        
        #lemmas = [token.lemma_ for token in docs]
        
        stems = flatten([nltk.word_tokenize(prop.lower(), language="french") for prop in data[row.name].dropna().values])
        stems = [stemmer.stem(w) for w in stems if w not in stops]
        #lemmas = flatten([nltk.pos_tag(nltk.word_tokenize(prop.lower(), language="french")) for prop in data[row.name].dropna().values])
        #lemmas = [word for (word, tag) in lemmas if tag not in set(".:()")]
        #if(i==1):
            #print(sorted(lemmas))
        row.unigram = Counter(ngrams(stems, 1)).most_common(10)
        row.bigram = Counter(ngrams(stems, 2)).most_common(10)
    else:
        row.propLen_moy = "question fermée"
        row.nb_phrases_moy = "question fermée"

        row.unigram = "question fermée"
        row.bigram = "question fermée"

    return row

In [ ]:
blob.tags

In [ ]:
corse = get_df(region_test)

In [ ]:
corse

In [ ]:
test2, data = init_df(region_test)

In [ ]:
test2

In [ ]:
from textblob import TextBlob
from textblob_fr import PatternTagger, PatternAnalyzer

In [ ]:
text = "en mon humble opinion personellement je trouve ça plutôt banale tout ça"
blob = TextBlob(text, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

In [ ]:
blob.tags

In [ ]:
blob.sentiment

In [ ]:
test2.iloc[1]["unigram"]

In [ ]:
full_summary, data = init_df(df)

In [ ]:
start = time()
full_summary = full_summary.parallel_apply(fill_row, axis=1)
print(f"Time: {time()-start:.2f}s")

In [ ]:
reg

In [ ]:
for region, group in reg.groups:
    print(region)

In [ ]:
for region in reg.groups:
    print(region)

In [ ]:
df[df.columns[-1]].dropna()

In [ ]:
for region in reg.groups:
    summary, data = init_df(reg.get_group(region))
    start = time()
    summary = summary.parallel_apply(fill_row, axis=1)
    summary.to_csv(f"data/eco{region}Summary.csv")
    print(f"Time: {time()-start:.2f}s")

In [ ]:
start = time()
test2 = test2.apply(fill_row, axis=1)
print(f"Time: {time()-start:.2f}s")

In [ ]:
full_summary.to_csv("data/ecoFullSummary.csv")

In [ ]:
"""
#Nombre de propositions
test.loc['nb_prop'] = ouvertes_only[ouvertes_only.columns[1]].shape[0]

#Chaque proposition sous forme de Doc spacy
test.loc['docs'] = [list(nlp.pipe(ouvertes_only[ouvertes_only.columns[1]].dropna()))]

#Liste de tokens de toutes les propositions combinées
test.loc['tokens'] = [filter_flatten_tags(test[test.columns[0]]['docs'])]

#POS tag + lemme pour chaque token
test.loc['POS'] = [[(token.lemma_, token.pos_) for token in test[test.columns[0]]['tokens']]]

#Longueur moyenne d'une proposition
test.loc['propLen_moy'] = sum([len(doc) for doc in test[test.columns[0]]['docs']])/test[test.columns[0]]['nb_prop']

#Nombre de phrases moyen pour une proposition
test.loc['nb_phrases_moy'] = sum([len(TextBlob(doc.text).sentences) for doc in test[test.columns[0]]['docs']])/test[test.columns[0]]['nb_prop']

#Unigram sur les tokens
test.loc['unigram'] = [list(ngrams([lemma for lemma, _ in test[test.columns[0]]['POS']], 1))]

#Bigram sur les tokens
test.loc['bigram'] =  [list(ngrams([lemma for lemma, _ in test[test.columns[0]]['POS']], 2))]
"""
print()

In [ ]:
unigrams = Counter(test[test.columns[0]]['unigram'])
bigrams =  Counter(test[test.columns[0]]['bigram'])

In [ ]:
unigrams.most_common(10)

In [ ]:
bigrams.most_common(10)

In [ ]:
start = time()
docs = list(nlp.pipe(ouvertes_only[ouvertes_only.columns[1]].dropna()))
print(f"Time: {time()-start:.2f}s")